# Weekly project 6
Today we will continue work from monday.
We will follow the style of last week.

Weekly project:
- You will need to implement your own k-means algorithm. You are not allowed to use the implementation in `sklearn`.
- It should be able to cluster each of the different figures.
- Extend your k-means so it finds the optimal amount of clusters.
  
## Challenge
- Implement the mean shift clustering algorithm


In [1]:

import copy

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d


def draw_labels_on_model(pcl, labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])


d = 4
mesh = o3d.geometry.TriangleMesh().create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh().create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh().create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh().create_torus().translate((-d, -d, 0))
# mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate((0, -d, 0))
# mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate((d, -d, 0))

## apply k means on this
point_cloud = mesh.sample_points_uniformly(int(2e3))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
import random as rand


def euclidean(a: np.array, b: np.array) -> float:
    return np.linalg.norm(a - b)


def find_center(points: np.array, index: list[int]) -> np.array:
    mean = np.mean(points[index, :], axis=0)
    return mean  #np.argmin([euclidean(points[i, :], mean) if i in index else euclidean(points[index[0], :], mean) + 1
    #     for i in range(points.shape[0])])


def create_label(length: int, clusters: list[list[int]]) -> np.array:
    labels = np.zeros((length))
    for i in range(len(clusters)):
        labels[clusters[i]] = i
    return labels


def kmeans(k, points, n_init=10, tol=1e-04, distance=euclidean) -> np.array:
    best_score = None
    best_clusters = None

    for _ in range(n_init):
        centers = rand.choices([points[i, :] for i in range(points.shape[0])], k=k)
        while True:
            clusters = [[] for _ in range(k)]

            for i in range(points.shape[0]):
                p = points[i, :]
                cluster_idx = np.argmin([distance(p, c) for c in centers])
                clusters[cluster_idx].append(i)

            new_centers = [find_center(points, cluster) for cluster in clusters]

            if all([distance(new_centers[i], centers[i]) < tol for i in range(k)]):
                score = 0
                for i, cluster in enumerate(clusters):
                    score += sum([distance(new_centers[i], points[idx]) for idx in cluster])
                score /= points.shape[0]
                # print(score)
                # labels = create_label(points.shape[0], clusters)
                # draw_labels_on_model(point_cloud, labels)

                if best_score is None or score < best_score:
                    best_score = score
                    best_clusters = clusters
                break
            centers = new_centers

    return create_label(points.shape[0], best_clusters)

In [16]:
labels = kmeans(4, np.asarray(point_cloud.points))

In [17]:
draw_labels_on_model(point_cloud, labels)

# mean shift clustering algo

In [71]:
one_over_sqrt_two_pi = 1 / np.sqrt(2 * np.pi)


def gaussian(x):
    return one_over_sqrt_two_pi * np.exp(-np.linalg.norm(x) ** 2 / 2)


def gaussian_opti(x):
    # x is a list of vector (np.array of dim (n of line, 3))
    return one_over_sqrt_two_pi * np.exp(-np.sum(np.square(x), axis=1) / 2)


def mean_shift(points, kernel=gaussian_opti, h=2, tol=1e-04) -> np.array:
    centers = points[:]
    while True:
        new_centers = np.zeros_like(centers)
        for i, c in enumerate(centers):
            cluster = points[np.linalg.norm(points - c, axis=1) < h, :]
            factor = kernel((cluster - c) / h)
            new_centers[i, :] = np.sum(factor[:, np.newaxis] * cluster, axis=0) / np.sum(factor)

        if np.mean(np.linalg.norm(new_centers - centers, axis=1)) < tol:
            break
        centers = new_centers

    labels = np.zeros((centers.shape[0], 1))
    unique_centers = []
    for i, c in enumerate(centers):
        label = -1

        for l, know_c in enumerate(unique_centers):
            if np.linalg.norm(know_c - c) < tol:
                label = l
                break

        if label != -1:
            labels[i] = label
        else:
            labels[i] = len(unique_centers)
            unique_centers.append(c)

In [72]:
label = mean_shift(np.asarray(point_cloud.points))

In [73]:
draw_labels_on_model(point_cloud, labels)